# Initializing Code

### Each of the boxes of code in this section should be run once (in the order they are here). 

To run a box, click on it (so you can see your cursor blinking inside the box) then press "cntrl+enter" (or click the "run" button at the top of this page).

In [1]:
from pickle import load
from keras.models import load_model
from keras.preprocessing.sequence import pad_sequences
import keras
from keras.preprocessing.text import Tokenizer
import numpy as np
import spacy
import requests
import tqdm

In [16]:
# define functions 
def read_file(filepath):
    
    with open(filepath) as f:
        str_text = f.read()
    
    return str_text

def separate_punc(doc_text):
    return [token.text.lower() for token in nlp(doc_text) if token.text not in '\n\n \n\n\n!"-#$%&()--.*+,-/:;<=>?@[\\]^_`{|}~\t\n ']

def generate_text(model, tokenizer, seq_len, seed_text, num_gen_words, threshold):
    '''
    INPUTS:
    model : model that was trained on text data
    tokenizer : tokenizer that was fit on text data
    seq_len : length of training sequence
    seed_text : raw string text to serve as the seed
    num_gen_words : number of words to be generated by model
    '''
    
    # Final Output
    output_text = []
    
    # Intial Seed Sequence
    input_text = seed_text
    
    # Confidendence (note this is generated by the model and may be over-confident; in the future may calibrate with a Bayesian NN)
    confidence = 1
    
    # Create num_gen_words
    for i in range(num_gen_words):
        
        # Take the input text string and encode it to a sequence
        encoded_text = tokenizer.texts_to_sequences([input_text])[0]

        # Pad sequences to our trained rate (50 words in the video)
        pad_encoded = pad_sequences([encoded_text], maxlen=seq_len, truncating='pre')

        # Predict Class Probabilities for each word
        pred_word_ind = model.predict_classes(pad_encoded, verbose=0)[0]
        wd_confi = model.predict(pad_encoded, verbose=0)[0].max()
        confidence = confidence*wd_confi

        # Make sure addition of this word won't push us out of our confidence level
        if confidence < threshold:
            confidence = confidence/wd_confi # adjust confidence for reporting
            break
        
        else:
        
            # Grab word
            pred_word = tokenizer.index_word[pred_word_ind] 

            # Update the sequence of input text (shifting one over with the new word)
            input_text += ' ' + pred_word

            output_text.append(pred_word)
        
    # Make it look like a sentence.
    confidence = round(confidence*100,2)
    print(f"Confidence is: {confidence}%")
    return ' '.join(output_text)

In [4]:
# Load spaCy NLP
nlp = spacy.load('en_core_web_sm',disable=['parser', 'tagger','ner'])
nlp.max_length = 1198623

# Load the concatenated letters for processing
d = read_file('concatenated_letters.txt')
tokens = separate_punc(d)

# organize into sequences of tokens
train_len = 50+1 # 50 training words , then one target word

# Empty list of sequences
text_sequences = []

for i in range(train_len, len(tokens)):
    
    # Grab train_len# amount of characters
    seq = tokens[i-train_len:i]
    
    # Add to list of sequences
    text_sequences.append(seq)
    
# integer encode sequences of words
tokenizer = Tokenizer()
tokenizer.fit_on_texts(text_sequences)
sequences = tokenizer.texts_to_sequences(text_sequences)

vocabulary_size = len(tokenizer.word_counts)

# Create Numpy Matrix
sequences = np.array(sequences)

# Initialize Matrices
X = sequences[:,:-1] # first 49 words in the sequence
y = sequences[:,-1] # last word in the sequence
seq_len = X.shape[1] # set sequence length for "generate_text" tool


# load trained model
model = load_model('Disraeli_bot1.h5')
tokenizer = load(open('Disraeli_bot1','rb'))

C:\Users\Paul\Anaconda3\lib\site-packages\spacy\util.py:275: UserWarning: [W031] Model 'en_core_web_sm' (2.2.0) requires spaCy v2.2 and is incompatible with the current spaCy version (2.3.5). This may lead to unexpected results or runtime errors. To resolve this, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


# This section lets you interact with the Disraeli neural net (a.k.a. "Disraeli-bot")

You can run the box below as many times as you want. When you run it, it will ask for a "prompt" - this is analogous to the first sentence (or two - you can type as much as you want, 25-50 words is ideal) you type in a Gmail email. Then, the Disraeli-bot will spit out up to the next 25 words that Gmail would suggest continuing with (the suggestion powered by the neural net trained on Disraeli's correspondence) provided the confidence level in the prediction does not fall below a specified level (I put the default at 50%, but you can tweak that by changing the "threshold=0.X" argument).

**A few notes for Nick** - this performs better on topics covered in the letters than out (I had some from 1868 and 1857 from https://www.jstor.org/stable/10.3138/j.ctt9qh93p). You probably understand much better what the content is, but it looks to me like this is a lot of personal correspondence, so perhaps not super topical for our purposes? Also of note - I didn't get a chance to clean the letters yet, so the footnote annotations and "EBSCO Host checked out to paul.connell@columbia.edu", etc. are, unfortunately, included in the nerual net. These are all things that we can fix easily, but take time. Also, the more letters we can feed it (I hit my max allowance of 100 pages/day) the better the performance will get.

In [60]:
seed_text= input("Give Disraeli-bot a prompt: ")
# seed_text = "Given the opportunity to purchase half of the outstanding equity in the Suez company, which the Pasha is eager for in order to retire debt, I think"
print("Disraeli-bot continues with:...")
generate_text(model,tokenizer,seq_len,seed_text=seed_text,num_gen_words=25, threshold=0.3
            )

Give Disraeli-bot a prompt: Russia and Turkey are at war. Our means of supporting the Ottomans are
Disraeli-bot continues with:...
Confidence is: 32.45%


'of any sentimental tomfoolery ®'

**Odds and Ends** Use the pip istalls if you don't already have the relevant packages downloaded to your programming environment. The last box includes code to download the model from GitHub if it is not already saved locally.

In [ ]:
# %pip install pandas
# %pip install keras
# %pip install spacy
# %pip install https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-3.0.0/en_core_web_sm-3.0.0-py3-none-any.whl
# %pip install https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-3.0.0/en_core_web_sm-3.0.0.tar.gz
# %pip install tensorflow

# %run -m spacy download en_core_web_sm

In [36]:
# # load trained model and tokenizer from GitHub
# url = "https://raw.githubusercontent.com/pjconnell/disraeli_neural_net/main/Disraeli_bot1.h5"
# page = requests.get(url)
# m = page.text
# # # load model
# model = load_model(m)

# # load tokenizer
# url = "https://raw.githubusercontent.com/pjconnell/disraeli_neural_net/main/Disraeli_bot1"
# page = requests.get(url)
# t = page.text
# tokenizer = load(open(t,'rb'))